<a href="https://colab.research.google.com/github/Angelin5/skills-introduction-to-github/blob/main/Vibeathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q faster-whisper sentence-transformers streamlit plotly yt-dlp pandas
!apt-get install -y ffmpeg > /dev/null 2>&1

print("✅ Dependencies installed!")

✅ Dependencies installed!


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

print("✅ Cloudflared installed!")

✅ Cloudflared installed!


In [ ]:
%%writefile app.py
"""
🎯 VIBEATHON 2024 - VIDEO RELEVANCE SCORER
Enhanced with Modern UI
"""

import os
import tempfile
import subprocess
import math
import re

import streamlit as st
import yt_dlp
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sentence_transformers import SentenceTransformer, util

try:
    from faster_whisper import WhisperModel
    FASTER_WHISPER = True
except Exception:
    FASTER_WHISPER = False

MODEL_CACHE_DIR = "/content/models"
os.makedirs(MODEL_CACHE_DIR, exist_ok=True)

PROMO_KEYWORDS = [
    "subscribe", "like", "comment", "bell icon", "notification",
    "sponsor", "patreon", "merch", "buy now", "discount code",
    "link in description", "check out", "visit our website",
    "special offer", "limited time", "exclusive deal"
]

CATEGORY_KEYWORDS = {
    "Technology": ["ai", "software", "coding", "programming", "tech", "computer", "algorithm", "machine learning"],
    "Education": ["learn", "tutorial", "course", "lesson", "study", "teach", "education", "training"],
    "Entertainment": ["funny", "comedy", "music", "movie", "game", "entertainment", "fun"],
    "Business": ["business", "marketing", "sales", "startup", "entrepreneur", "finance", "investment"],
    "Health": ["health", "fitness", "workout", "nutrition", "wellness", "medical", "exercise"],
    "Science": ["science", "research", "experiment", "discovery", "theory", "physics", "chemistry"]
}

def load_custom_css():
    st.markdown("""
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800&display=swap');

        * {
            font-family: 'Inter', sans-serif;
        }

        /* Remove default Streamlit elements */
        #MainMenu {visibility: hidden;}
        footer {visibility: hidden;}
        header {visibility: hidden;}

        /* Main background - Dark gradient */
        .stApp {
            background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%) !important;
        }

        /* Top header bar - make it dark */
        header[data-testid="stHeader"] {
            background: #1a1a2e !important;
        }

        /* Top toolbar area */
        .main > div:first-child {
            background: #1a1a2e !important;
        }

        .main .block-container {
            padding-top: 2rem;
            max-width: 1200px;
        }

        /* Header styling */
        .main-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 3rem 2rem;
            border-radius: 20px;
            color: white;
            text-align: center;
            margin-bottom: 2rem;
            box-shadow: 0 15px 40px rgba(102, 126, 234, 0.4);
        }
        .main-header h1 {
            margin: 0;
            font-size: 2.8rem;
            font-weight: 800;
        }
        .main-header p {
            margin: 0.8rem 0 0 0;
            font-size: 1.1rem;
        }

        /* Success banner */
        .success-banner {
            background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
            color: white;
            padding: 1.8rem;
            border-radius: 15px;
            font-weight: 700;
            text-align: center;
            font-size: 1.3rem;
            margin: 1.5rem 0;
            box-shadow: 0 10px 25px rgba(0,0,0,0.3);
        }

        /* Info boxes */
        .info-box {
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            border-left: 6px solid #2196f3;
            padding: 1.5rem;
            border-radius: 12px;
            margin: 1.5rem 0;
            font-weight: 600;
            color: #e0e0e0;
            box-shadow: 0 8px 20px rgba(0,0,0,0.3);
        }

        .music-box {
            background: linear-gradient(135deg, #3a2a4a 0%, #4a2a5a 100%);
            border-left: 6px solid #9c27b0;
            padding: 1.5rem;
            border-radius: 12px;
            margin: 1.5rem 0;
            font-weight: 600;
            color: #e0e0e0;
            box-shadow: 0 8px 20px rgba(0,0,0,0.3);
        }

        /* Category badges */
        .category-badge {
            display: inline-block;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 0.7rem 1.5rem;
            border-radius: 25px;
            margin: 0.4rem;
            font-weight: 700;
            box-shadow: 0 6px 15px rgba(102, 126, 234, 0.4);
        }

        /* Explanation text */
        .explanation-text {
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            padding: 2rem;
            border-radius: 15px;
            box-shadow: 0 6px 20px rgba(0,0,0,0.3);
            line-height: 1.8;
            font-size: 1.05rem;
            color: #e0e0e0;
            border: 1px solid #444;
        }

        /* Metrics styling */
        div[data-testid="metric-container"] {
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            padding: 1.8rem;
            border-radius: 15px;
            box-shadow: 0 8px 25px rgba(0,0,0,0.3);
            border: 2px solid #444;
        }

        div[data-testid="metric-container"] [data-testid="stMetricValue"] {
            font-size: 3rem;
            font-weight: 800;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;

        }

        div[data-testid="metric-container"] [data-testid="stMetricLabel"] {
            color: #b0b0b0 !important;
        }

        /* Button styling */
        .stButton>button {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 1rem 3rem;
            font-size: 1.2rem;
            font-weight: 700;
            border-radius: 15px;
            border: none;
            width: 100%;
            box-shadow: 0 8px 25px rgba(102, 126, 234, 0.4);
            text-transform: uppercase;
            transition: all 0.3s ease;
        }

        .stButton>button:hover {
            transform: translateY(-2px);
            box-shadow: 0 12px 30px rgba(102, 126, 234, 0.6);
        }

        /* Sidebar styling */
        section[data-testid="stSidebar"] {
            background: linear-gradient(180deg, #1a1a2e 0%, #16213e 100%);
            border-right: 1px solid #444;
        }

        section[data-testid="stSidebar"] * {
            color: #e0e0e0 !important;
        }

        section[data-testid="stSidebar"] .stMarkdown {
            color: #e0e0e0 !important;
        }

        /* Tabs styling */
        .stTabs [data-baseweb="tab-list"] {
            background: transparent;
        }

        .stTabs [data-baseweb="tab"] {
            height: 4rem;
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            border-radius: 15px 15px 0 0;
            padding: 0 2.5rem;
            font-weight: 700;
            color: #b0b0b0;
            border: 1px solid #444;
        }

        .stTabs [aria-selected="true"] {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white !important;
        }

        /* Input fields */
        .stTextInput>div>div>input,
        .stTextArea>div>div>textarea {
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            color: #e0e0e0;
            border: 1px solid #444;
            border-radius: 10px;
        }

        /* Radio buttons */
        .stRadio>div {
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            padding: 1rem;
            border-radius: 10px;
            border: 1px solid #444;
        }

        /* Expander */
        .streamlit-expanderHeader {
            background: linear-gradient(135deg, #2a2d5a 0%, #1e3a5f 100%);
            color: #e0e0e0;
            border-radius: 10px;
            border: 1px solid #444;
        }

        /* All text elements */
        .stMarkdown, p, span, label {
            color: #e0e0e0 !important;
        }

        h1, h2, h3, h4, h5, h6 {
            color: #ffffff !important;
        }

        /* Subheaders */
        .stSubheader {
            color: #ffffff !important;
            font-weight: 700;
        }

    </style>
    """, unsafe_allow_html=True)

def download_youtube_audio(url, out_dir=None):
    if out_dir is None:
        out_dir = tempfile.mkdtemp()
    outtmpl = os.path.join(out_dir, "audio.%(ext)s")
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": outtmpl,
        "noplaylist": True,
        "quiet": True,
        "no_warnings": True,
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "wav", "preferredquality": "192"}],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
        audio_path = os.path.splitext(filename)[0] + ".wav"
        title = info.get("title", "")
        description = info.get("description", "")
    return audio_path, title, description

def extract_audio_from_video(video_path, out_dir=None):
    if out_dir is None:
        out_dir = tempfile.mkdtemp()
    audio_path = os.path.join(out_dir, "extracted_audio.wav")
    cmd = ["ffmpeg", "-y", "-i", video_path, "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", audio_path]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"FFmpeg failed: {result.stderr}")
    return audio_path

def probe_audio_for_music(audio_path, probe_seconds=4):
    probe_tf = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    probe_path = probe_tf.name
    probe_tf.close()
    cmd = ["ffmpeg", "-y", "-i", audio_path, "-t", str(probe_seconds), "-ac", "1", "-ar", "16000", "-f", "wav", probe_path]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    try:
        import wave
        wf = wave.open(probe_path, "rb")
        frames = wf.readframes(wf.getnframes())
        wf.close()
        if len(frames) == 0:
            os.remove(probe_path)
            return False
        samples = np.frombuffer(frames, dtype=np.int16).astype(np.float32) / 32768.0
        if samples.size < 1024:
            os.remove(probe_path)
            return False
        zcr = ((samples[:-1] * samples[1:]) < 0).sum() / max(1, samples.size - 1)
        fft = np.abs(np.fft.rfft(samples))
        freqs = np.fft.rfftfreq(samples.size, d=1/16000.0)
        centroid = (freqs * fft).sum() / (fft.sum() + 1e-9)
        music_score = (centroid / 8000.0 * 0.7) + ((1 - zcr) * 0.3)
        os.remove(probe_path)
        return music_score > 0.25
    except:
        try:
            os.remove(probe_path)
        except:
            pass
        return False

def detect_promotional_content(text):
    text_lower = text.lower()
    found = [kw for kw in PROMO_KEYWORDS if kw in text_lower]
    return len(found) > 0, found

def categorize_content(text):
    text_lower = text.lower()
    scores = {}
    for category, keywords in CATEGORY_KEYWORDS.items():
        count = sum(1 for kw in keywords if kw in text_lower)
        if count > 0:
            scores[category] = count
    if not scores:
        return ["General"]
    sorted_cats = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [cat for cat, _ in sorted_cats[:2]]

@st.cache_resource
def load_whisper_model_faster(model_name="distil-large-v3"):
    if not FASTER_WHISPER:
        raise RuntimeError("faster_whisper not installed")
    return WhisperModel(model_name, device="cpu", compute_type="int8")

@st.cache_resource
def load_embedder(model_name="all-MiniLM-L6-v2"):
    return SentenceTransformer(model_name)

def transcribe_faster_whisper(model, audio_path, language=None, is_music=False):
    beam_size = 3 if is_music else 1
    segments, _ = model.transcribe(audio_path, language=language, beam_size=beam_size, vad_filter=not is_music)
    out_segments = []
    texts = []
    for s in segments:
        txt = s.text.strip()
        out_segments.append({"start": getattr(s, "start", 0.0), "end": getattr(s, "end", 0.0), "text": txt})
        texts.append(txt)
    return out_segments, " ".join(texts).strip()

def clean_lyrics_text(raw):
    text = raw.strip()
    text = re.sub(r"\b(\w+)(\s+\1\b)+", r"\1", text, flags=re.IGNORECASE)
    text = re.sub(r"([.!?])\s+", r"\1\n", text)
    text = re.sub(r"[ \t]{2,}", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)
    return text

def chunk_segments_by_time(segments, chunk_duration=15.0):
    if not segments:
        return []
    total_end = segments[-1].get("end", 0.0)
    n_chunks = max(1, math.ceil(total_end / chunk_duration))
    chunks = []
    for i in range(n_chunks):
        start_t = i * chunk_duration
        end_t = (i + 1) * chunk_duration
        texts = [s.get("text", "").strip() for s in segments if s.get("start", 0) < end_t and s.get("end", 0) > start_t]
        chunk_text = " ".join(texts).strip()
        is_promo, promo_kws = detect_promotional_content(chunk_text)
        chunks.append({"index": i, "start": start_t, "end": end_t, "text": chunk_text, "is_promotional": is_promo, "promo_keywords": promo_kws})
    return chunks

def compute_relevance(title_desc_text, chunks, embedder):
    chunk_texts = [c["text"] if c["text"] else "" for c in chunks]
    if title_desc_text and title_desc_text.strip():
        query_emb = embedder.encode(title_desc_text, convert_to_tensor=True)
        chunk_embs = embedder.encode(chunk_texts, convert_to_tensor=True)
        sims = util.cos_sim(query_emb, chunk_embs).cpu().numpy().flatten()
    else:
        chunk_embs = embedder.encode(chunk_texts, convert_to_tensor=True)
        avg = np.mean(chunk_embs.cpu().numpy(), axis=0)
        sims = util.cos_sim(avg, chunk_embs).cpu().numpy().flatten()
    sims_pct = ((sims + 1.0) / 2.0 * 100.0).tolist()
    weights = np.array([max(1, len(c["text"].split()) if c["text"] else 0) for c in chunks], dtype=float)
    sims_arr = np.array(sims_pct, dtype=float)
    overall = float(np.sum(sims_arr * weights) / np.sum(weights)) if weights.sum() > 0 else float(np.mean(sims_arr))
    overall = max(0.0, min(100.0, overall))
    per_chunk = []
    for i, c in enumerate(chunks):
        score = round(float(sims_pct[i]) if i < len(sims_pct) else 0.0, 2)
        if c["is_promotional"]:
            score = score * 0.7
        per_chunk.append({"index": c["index"], "start": c["start"], "end": c["end"], "text": c["text"],
                         "similarity_pct": round(score, 2), "is_promotional": c["is_promotional"],
                         "promo_keywords": c["promo_keywords"]})
    return round(overall, 2), per_chunk

def generate_detailed_explanation(overall_score, per_chunk, title_desc):
    low_chunks = [c for c in per_chunk if c["similarity_pct"] < 40]
    promo_chunks = [c for c in per_chunk if c["is_promotional"]]
    high_chunks = [c for c in per_chunk if c["similarity_pct"] >= 70]
    expl = []
    title_preview = title_desc[:50] + "..." if len(title_desc) > 50 else title_desc
    if overall_score >= 80:
        expl.append(f"✅ Strong Match ({overall_score}%): Content aligns with '{title_preview}'")
    elif overall_score >= 50:
        expl.append(f"⚠️ Partial Match ({overall_score}%): Video partially relates to '{title_preview}'")
    else:
        expl.append(f"❌ Poor Match ({overall_score}%): Content diverges from '{title_preview}'")
    if high_chunks:
        expl.append(f"• {len(high_chunks)}/{len(per_chunk)} segments highly relevant")
    if low_chunks:
        expl.append(f"• {len(low_chunks)} segments off-topic")
    if promo_chunks:
        promo_kw_set = set()
        for c in promo_chunks:
            promo_kw_set.update(c["promo_keywords"])
        expl.append(f"• {len(promo_chunks)} promotional segments ({', '.join(list(promo_kw_set)[:3])})")
    return " ".join(expl)

def create_heatmap(per_chunk):
    if not per_chunk:
        return None
    df = pd.DataFrame([{'Time': f"{c['start']:.0f}s-{c['end']:.0f}s", 'Start': c['start'], 'Relevance': c['similarity_pct'],
                        'Status': '🔴 Promotional' if c['is_promotional'] else ('🟢 Relevant' if c['similarity_pct'] >= 60 else '🟡 Low'),
                        'Preview': c['text'][:100] + '...' if len(c['text']) > 100 else c['text']} for c in per_chunk])
    colors = ['#FF4444' if c['is_promotional'] else '#00E676' if c['similarity_pct'] >= 70 else '#FFD700' if c['similarity_pct'] >= 40 else '#FF6B6B' for c in per_chunk]
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df['Start'], y=df['Relevance'], marker=dict(color=colors), text=[f"{r:.1f}%" for r in df['Relevance']],
                         textposition='outside', hovertemplate='<b>%{customdata[0]}</b><br>Relevance: <b>%{y:.1f}%</b><br>Status: %{customdata[1]}<extra></extra>',
                         customdata=df[['Time', 'Status']].values))
    fig.update_layout(title={'text': '📊 Video Relevance Heatmap', 'x': 0.5, 'font': {'size': 22}},
                     xaxis_title='Time (seconds)', yaxis_title='Relevance (%)', yaxis_range=[0, 105], height=450, showlegend=False)
    return fig

st.set_page_config(page_title="Video Relevance Scorer", layout="wide", page_icon="🎯")
load_custom_css()

st.markdown('<div class="main-header"><h1>🎯 AI Video Relevance Scorer</h1><p>Detect Clickbait • Analyze Quality • Identify Promo</p><p style="font-size:0.9rem;margin-top:0.5rem;">Vibeathon 2024 | Whisper AI & Sentence Transformers</p></div>', unsafe_allow_html=True)

with st.sidebar:
    st.header("⚙️ Settings")
    chunk_duration = st.slider("Chunk Duration (s)", 5.0, 60.0, 15.0, 5.0)
    embedding_model = st.selectbox("Embedding Model", ["all-MiniLM-L6-v2", "paraphrase-MiniLM-L6-v2"])
    st.markdown("---")
    st.markdown("### 📊 Features\n- ✅ YouTube & Upload\n- ✅ Music Detection\n- ✅ Promo Detection\n- ✅ Interactive Heatmap\n- ✅ Categorization")

tab1, tab2 = st.tabs(["🎬 Analyze", "📖 Docs"])

with tab1:
    col1, col2 = st.columns([1, 1])
    with col1:
        st.subheader("Input Method")
        input_method = st.radio("Choose:", ["YouTube URL", "Upload Video File"])
        youtube_url = ""
        uploaded_file = None
        if input_method == "YouTube URL":
            youtube_url = st.text_input("Enter YouTube URL", placeholder="https://youtube.com/watch?v=...")
        else:
            uploaded_file = st.file_uploader("Upload video", type=["mp4", "mov", "avi", "mkv", "webm"])
        st.markdown("---")
        title_input = st.text_input("Title (optional)")
        desc_input = st.text_area("Description (optional)", height=100)
        language = st.text_input("Language (e.g., 'en')", placeholder="auto")
        run_button = st.button("🚀 Analyze Video", type="primary", use_container_width=True)
    with col2:
        results_container = st.container()

if run_button:
    with results_container:
        if input_method == "YouTube URL" and not youtube_url.strip():
            st.error("Please provide a YouTube URL")
            st.stop()
        elif input_method == "Upload Video File" and uploaded_file is None:
            st.error("Please upload a video file")
            st.stop()
        try:
            if input_method == "YouTube URL":
                with st.spinner("📥 Downloading..."):
                    audio_path, yt_title, yt_desc = download_youtube_audio(youtube_url)
            else:
                with st.spinner("📥 Extracting..."):
                    temp_dir = tempfile.mkdtemp()
                    video_path = os.path.join(temp_dir, uploaded_file.name)
                    with open(video_path, "wb") as f:
                        f.write(uploaded_file.read())
                    audio_path = extract_audio_from_video(video_path, temp_dir)
                    yt_title = ""
                    yt_desc = ""
            with st.spinner("🎵 Detecting..."):
                is_music = probe_audio_for_music(audio_path)
            if is_music:
                st.markdown('<div class="music-box">🎵 Music detected</div>', unsafe_allow_html=True)
            else:
                st.markdown('<div class="info-box">🎤 Speech detected</div>', unsafe_allow_html=True)
            with st.spinner("🤖 Loading models..."):
                whisper_model = load_whisper_model_faster()
                embedder = load_embedder(embedding_model)
            with st.spinner("✍️ Transcribing..."):
                segments, raw_transcript = transcribe_faster_whisper(whisper_model, audio_path, language=language if language else None, is_music=is_music)
            transcript = clean_lyrics_text(raw_transcript) if is_music else raw_transcript
            with st.spinner("📊 Computing..."):
                chunks = chunk_segments_by_time(segments, chunk_duration)
                title_desc = ((title_input.strip() or yt_title) + " " + (desc_input.strip() or yt_desc)).strip()
                overall_score, per_chunk = compute_relevance(title_desc, chunks, embedder)
                explanation = generate_detailed_explanation(overall_score, per_chunk, title_desc or "content")
                categories = categorize_content(transcript)
            st.markdown('<div class="success-banner">✅ Analysis Complete!</div>', unsafe_allow_html=True)
            col_a, col_b = st.columns([1, 2])
            with col_a:
                st.metric("Overall Score", f"{overall_score}%")
                st.markdown(f"**Categories:** {' '.join([f'<span class=\"category-badge\">{cat}</span>' for cat in categories])}", unsafe_allow_html=True)
            with col_b:
                st.markdown(f'<div class="explanation-text">{explanation}</div>', unsafe_allow_html=True)
            st.markdown("---")
            st.subheader("📈 Heatmap")
            fig = create_heatmap(per_chunk)
            if fig:
                st.plotly_chart(fig, use_container_width=True)
            with st.expander("📄 Transcript", expanded=False):
                st.text_area("", transcript, height=300)
            st.markdown("---")
            st.subheader("⚠️ Flagged Segments")
            flagged = [c for c in per_chunk if c["similarity_pct"] < 40 or c["is_promotional"]]
            if flagged:
                for c in flagged:
                    status = "🔴 Promo" if c["is_promotional"] else "🟡 Low"
                    preview = c["text"][:150] + "..." if len(c["text"]) > 150 else c["text"]
                    st.markdown(f"**Seg {c['index']}** ({c['start']:.1f}s-{c['end']:.1f}s) | {status} | {c['similarity_pct']}%\n> {preview}")
            else:
                st.success("No problematic segments!")
        except Exception as e:
            st.error(f"Error: {str(e)}")

with tab2:
    st.markdown("""
    ## 📖 How It Works

    ### Analysis Pipeline
    1. **Audio Extraction** - Converts video to audio
    2. **Content Detection** - Identifies music vs speech
    3. **Transcription** - Whisper AI speech-to-text
    4. **Semantic Analysis** - Embeddings comparison
    5. **Scoring** - 0-100% relevance per segment
    6. **Detection** - Flags promotional content

    ### Scoring Guide
    - **80-100%**: Strong match
    - **50-79%**: Partial match
    - **0-49%**: Poor match

    ### Tech Stack
    - Whisper (distil-large-v3)
    - Sentence Transformers
    - Plotly visualization
    - Streamlit UI
    """)

st.markdown("---")
st.markdown("*Built for Vibeathon 2024*")

Writing app.py


In [ ]:
# ====================
# CELL 4: Run Streamlit with Cloudflared
# ====================
import subprocess
import threading
import time
import re

def run_streamlit():
    """Run Streamlit in background"""
    subprocess.Popen(
        ["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=true"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

def run_cloudflared():
    """Run cloudflared and capture the URL"""
    process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:8501"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True
    )

    for line in process.stdout:
        print(line.strip())
        # Look for the trycloudflare.com URL
        if "trycloudflare.com" in line:
            match = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
            if match:
                url = match.group(0)
                print(f"\n{'='*60}")
                print(f"🚀 YOUR APP IS READY!")
                print(f"{'='*60}")
                print(f"\n🔗 Access your app at:\n{url}\n")
                print(f"{'='*60}\n")

# Start Streamlit
print("🚀 Starting Streamlit...")
threading.Thread(target=run_streamlit, daemon=True).start()

# Wait for Streamlit to start
time.sleep(5)

# Start Cloudflared
print("🌐 Creating public tunnel with Cloudflared...")
print("⏳ Please wait for the URL (may take 10-30 seconds)...\n")
run_cloudflared()


🚀 Starting Streamlit...
🌐 Creating public tunnel with Cloudflared...
⏳ Please wait for the URL (may take 10-30 seconds)...

2025-11-17T12:45:24Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-17T12:45:24Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-17T12:45:29Z INF +--------------------------------------------------------------------------------------------+
2025-11-17T12:45:29Z INF |  Your quick Tunnel has been created! Visit it at (

KeyboardInterrupt: 